In [1]:
#!pip install pyspark
#!pip install findspark
#!pip install setuptools
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import setuptools

In [2]:
findspark.init()

In [3]:
spark=SparkSession.builder.getOrCreate()
spark

In [ ]:
#spark.stop()

## Base Contratos Carnê

In [6]:
# Lendo o arquivo CSV
# Base Contratos Carne

df_carne = spark.read.csv("C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos Carne/banco de dados/contratos_carne.csv",sep=";", header=True, inferSchema=True)


In [7]:
df_carne.printSchema()

root
 |-- anomes: string (nullable = true)
 |-- cpf_cnpj: long (nullable = true)
 |-- conceito: integer (nullable = true)
 |-- id_contrato: long (nullable = true)
 |-- id_financeira: integer (nullable = true)
 |-- data: string (nullable = true)
 |-- id_agente: string (nullable = true)
 |-- dias_carencia: integer (nullable = true)
 |-- taxajuros: double (nullable = true)
 |-- cet: double (nullable = true)
 |-- saldo_contabil: double (nullable = true)
 |-- dias_maior_atraso: integer (nullable = true)
 |-- dias_maior_atraso_aberto: integer (nullable = true)
 |-- media_atraso: integer (nullable = true)
 |-- qtd_parcelas: integer (nullable = true)
 |-- qtd_parcelas_abertas: integer (nullable = true)
 |-- qtd_parcelas_pagas: integer (nullable = true)
 |-- valor_entrada: double (nullable = true)
 |-- valor_pago: double (nullable = true)
 |-- valor_pago_atraso: double (nullable = true)
 |-- valor_parcela: string (nullable = true)
 |-- data_vencimento_prox_parcela: string (nullable = true)
 |--

In [8]:
#df_202504=df.filter(col("anomes") == "2025/04") #Filtrar dados pelo mês de abril de 2025
#df_202504=spark.read.csv("C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos Carne/dados_behavior_2025-04-30.csv",sep=";", header=True, inferSchema=True)
#df = df_202504

#Tipos
tipos = {
    "cpf_cnpj": "double"#,
}

# conversão
for col_nome, tipo in tipos.items():
    df = df.withColumn(col_nome, col(col_nome).cast(tipo))
#df.show()

In [9]:
# info
df.printSchema()

root
 |-- anomes: string (nullable = true)
 |-- cpf_cnpj: double (nullable = true)
 |-- conceito: integer (nullable = true)
 |-- id_contrato: long (nullable = true)
 |-- id_financeira: integer (nullable = true)
 |-- data: string (nullable = true)
 |-- id_agente: string (nullable = true)
 |-- dias_carencia: integer (nullable = true)
 |-- taxajuros: double (nullable = true)
 |-- cet: double (nullable = true)
 |-- saldo_contabil: double (nullable = true)
 |-- dias_maior_atraso: integer (nullable = true)
 |-- dias_maior_atraso_aberto: integer (nullable = true)
 |-- media_atraso: integer (nullable = true)
 |-- qtd_parcelas: integer (nullable = true)
 |-- qtd_parcelas_abertas: integer (nullable = true)
 |-- qtd_parcelas_pagas: integer (nullable = true)
 |-- valor_entrada: double (nullable = true)
 |-- valor_pago: double (nullable = true)
 |-- valor_pago_atraso: double (nullable = true)
 |-- valor_parcela: string (nullable = true)
 |-- data_vencimento_prox_parcela: string (nullable = true)
 |

In [10]:
# números de linhas
df.count()

1525091

In [11]:
# filtro dias_maior_atraso_aberto>60
df.filter(col("dias_maior_atraso_aberto") > 60).show()


+-------+---------+--------+-----------+-------------+----------+--------------------+-------------+---------+-----+--------------+-----------------+------------------------+------------+------------+--------------------+------------------+-------------+----------+-----------------+-------------+----------------------------+------------------------------+---------------+--------------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+-------------------------

In [12]:
df_cpf=df.filter(col("cpf_cnpj") == 929085)
df_cpf.show(100)
## salvar arquivo
#df_cpf_p = df_cpf.toPandas() # pandas
#df_cpf_p.to_excel("df_cpf_p.xlsx", index=False)

+-------+--------+--------+-----------+-------------+----------+--------------------+-------------+---------+----+--------------+-----------------+------------------------+------------+------------+--------------------+------------------+-------------+----------+-----------------+-------------+----------------------------+------------------------------+---------------+--------------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+---------------------------

In [ ]:
# Qtd de contratos por Cpf
res = df.groupBy("cpf_cnpj").agg(count("id_contrato").alias("num_id_contrato"),
                                 max("dias_maior_atraso_aberto").alias("id_inad")) 
res.show()
res.filter((col("num_id_contrato") > 10) &
           (col("id_inad") > 1)).show()

+---------------+---------------+-------+
|       cpf_cnpj|num_id_contrato|id_inad|
+---------------+---------------+-------+
|2.3078760001E12|             10|      0|
|   2.78621007E8|              6|      0|
|    3.4384995E8|              2|      0|
|   3.92299984E8|              2|      0|
|   3.95892007E8|              3|      0|
|   4.01227944E8|              4|      0|
|   4.08728663E8|              2|      0|
|   4.44622985E8|              6|      0|
|   4.56979905E8|             11|      0|
|   4.97589931E8|             11|      0|
|   5.17977982E8|              1|      0|
|   5.19314948E8|              1|      0|
|   5.35693931E8|              2|      0|
|    5.6598599E8|              2|    399|
|   5.73103941E8|              2|     36|
|   6.56971975E8|              9|      0|
|   8.24364953E8|              1|      0|
|   8.40852231E8|              3|    160|
|    9.1334292E8|              4|     78|
|   9.17704916E8|              2|      0|
+---------------+---------------+-

In [23]:
#Agrupar
df.groupBy("conceito").count().show(truncate=False)

+--------+------+
|conceito|count |
+--------+------+
|26      |22551 |
|1       |149445|
|52      |30    |
|20      |206666|
|92      |6062  |
|4       |270054|
|100     |26    |
|23      |14526 |
|51      |518   |
|10      |151079|
|50      |49    |
|11      |7485  |
|99      |7448  |
|58      |24    |
|101     |10    |
+--------+------+



In [24]:
## Categorias
df.select("flag_renegociacao").distinct().show(truncate=False)

+-----------------+
|flag_renegociacao|
+-----------------+
|SIM              |
|NAO              |
+-----------------+



In [13]:
df.filter((col("id_agente") == " 033 VENDAS ATACADO")).show()

+-------+------------------+--------+-----------+-------------+----------+-------------------+-------------+---------+---+--------------+-----------------+------------------------+------------+------------+--------------------+------------------+-------------+----------+-----------------+-------------+----------------------------+------------------------------+---------------+--------------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+-------------------

In [14]:
# Excluindo linhas
df = df.filter(
    ~(
        (col("id_agente") == " 033 VENDAS ATACADO") 
    )
)
df.count()

1525086

In [21]:
df=df.withColumn('id_agente', trim(col('id_agente')))

In [23]:
df.groupBy("id_agente").count().show(truncate=False)

+----------------------------------+------+
|id_agente                         |count |
+----------------------------------+------+
|001 CREDIARIO                     |743328|
|004 KREDILIG (CDC)                |383220|
|011 RENEGOCIACAO CREDIARIO        |29034 |
|009 SAQUE EMERGENCIAL             |1722  |
|021 RENEGOCIACAO CREDIARIO (AG 11)|14533 |
|014 RENEGOCIACAO CDC              |28147 |
|025 RENEGOCIACAO CP (AG 15)       |16866 |
|015 RENEGOCIACAO CP               |30944 |
|019 RENEGOCIACAO SAQUE EMERGENCIAL|235   |
|005 KREDILIG (CP)                 |261274|
|029 RENEGOCIACAO SAQUE EMER (AG19)|127   |
|024 RENEGOCIACAO CDC (AG 14)      |15041 |
|044 SEGURO - KREDILIG (CDC)       |610   |
|033 VENDAS ATACADO                |5     |
+----------------------------------+------+



In [17]:
# tabela de frequencia
col_cat_df=['conceito','flag_renegociacao', 'id_agente', 'anomes']

# loop
for col_nome in col_cat_df:
    print(f"\n### Frequência para: {col_nome} ###")
    df.groupBy(col_nome).count().orderBy("count", ascending=False).show(truncate=False)


### Frequência para: conceito ###
+--------+------+
|conceito|count |
+--------+------+
|1       |407726|
|4       |392537|
|10      |347484|
|20      |277835|
|26      |31487 |
|23      |19932 |
|92      |18698 |
|11      |17414 |
|99      |10491 |
|51      |899   |
|50      |188   |
|58      |149   |
|100     |134   |
|101     |67    |
|52      |45    |
+--------+------+


### Frequência para: flag_renegociacao ###
+-----------------+-------+
|flag_renegociacao|count  |
+-----------------+-------+
|NAO              |1390159|
|SIM              |134927 |
+-----------------+-------+


### Frequência para: id_agente ###
+-----------------------------------+------+
|id_agente                          |count |
+-----------------------------------+------+
| 001 CREDIARIO                     |413702|
|001 CREDIARIO                      |329626|
|004 KREDILIG (CDC)                 |205283|
| 004 KREDILIG (CDC)                |177937|
| 005 KREDILIG (CP)                 |175141|
|005 KREDILIG

In [ ]:
# armazenar as tabelas
tabelas_frequencia_df1 = {}

for col_nome in col_cat_df:
    df_freq1 = df.groupBy(col_nome).count().orderBy("count", ascending=False)
    tabelas_frequencia_df1[col_nome] = df_freq1

'''# Excel
with pd.ExcelWriter("tab_freq_base_contratos_carnes.xlsx", engine="xlsxwriter") as writer:
    for nome, df_spark in tabelas_frequencia_df1.items():
        df_pandas = df_spark.toPandas() # pandas
        aba = nome[:31] if len(nome) > 31 else nome # para variáveis com nomes grandes 
        df_pandas.to_excel(writer, sheet_name=aba, index=False)'''

In [24]:
# Contar distintos

# loop ## não considera o valor nulo como distinto
df.select([
    countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns
]).show(truncate=False)

+---------------+-----------------+-----------------+--------------------+----------------------+-------------+------------------+----------------------+------------------+------------+-----------------------+--------------------------+---------------------------------+---------------------+---------------------+-----------------------------+---------------------------+----------------------+-------------------+--------------------------+----------------------+-------------------------------------+---------------------------------------+------------------------+-----------------------------+--------------------------+--------------------------------------+------------------------------------------+------------------------------------------+-------------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------------+----------------------------------------------+-----------------------

In [25]:
### Contar nulos
#df.select(count(when(col("data_vencimento_prox_parcela").isNull(), 1)).alias("nulos")).show()

#loop
df.select([
    count(when(col(c).isNull(), 1)).alias(f"{c}_nulos") for c in df.columns
]).show(truncate=False)

+------------+--------------+--------------+-----------------+-------------------+----------+---------------+-------------------+---------------+---------+--------------------+-----------------------+------------------------------+------------------+------------------+--------------------------+------------------------+-------------------+----------------+-----------------------+-------------------+----------------------------------+------------------------------------+---------------------+--------------------------+-----------------------+-----------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------

## Base Cadastro

In [33]:
## Base cartões

df2=spark.read.csv("C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base cartões/behavior.csv",header=True,inferSchema=True, sep=";") 

#Tipos
tiposdf2 = {
    "cpf_cnpj": "double"#,
}

# conversão
for col_nome, tipo in tiposdf2.items():
    df2 = df2.withColumn(col_nome, col(col_nome).cast(tipo))

df2.show()

+-------------------+--------------+--------------------+---------------+-----------+----+-----+-------------+--------------------+--------------------+--------------+------------+-------------+-------------+------------------+-------------+--------------------+------+--------+-----------------------------+--------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+---------------+------------------+
|  ANO_MES_MOVIMENTO|DATA_MOVIMENTO|                NOME|       cpf_cnpj|TIPO_PESSOA|SEXO|IDADE| ESTADO_CIVIL|   GRAU_ESCOLARIDADE|   NATUREZA_OCUPACAO|RENDA_DIGITADA|RENDA_VALIDA|NACIONALIDADE|UF_NASCIMENTO| CIDA

In [34]:
# info
df2.printSchema()

root
 |-- ANO_MES_MOVIMENTO: timestamp (nullable = true)
 |-- DATA_MOVIMENTO: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- cpf_cnpj: double (nullable = true)
 |-- TIPO_PESSOA: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- ESTADO_CIVIL: string (nullable = true)
 |-- GRAU_ESCOLARIDADE: string (nullable = true)
 |-- NATUREZA_OCUPACAO: string (nullable = true)
 |-- RENDA_DIGITADA: string (nullable = true)
 |-- RENDA_VALIDA: string (nullable = true)
 |-- NACIONALIDADE: string (nullable = true)
 |-- UF_NASCIMENTO: string (nullable = true)
 |-- CIDADE_NASCIMENTO: string (nullable = true)
 |-- UF_RESIDENCIA: string (nullable = true)
 |-- CIDADE_RESIDENCIA: string (nullable = true)
 |-- FILIAL: string (nullable = true)
 |-- REGIONAL: integer (nullable = true)
 |-- TEMPO_RELACIONAMENTO_KREDILIG: string (nullable = true)
 |-- TEMPO_INATIVIDADE_KREDILIG: string (nullable = true)
 |-- BLACKLIST: string (nullable = true)
 

### Duplos

In [25]:
## Removendo dados duplicados
duplos=df2.groupBy(col("CPF_CNPJ")).count().filter(col("count")>1)
duplos.show()
print(duplos.count())
df2.filter(col("CPF_CNPJ")=="1893026078").show()


+---------------+-----+
|       CPF_CNPJ|count|
+---------------+-----+
|9.1779553072E10|    2|
|  2.390921981E9|    2|
|  8.366555917E9|    2|
|7.5069725904E10|    2|
|1.0114464995E10|    2|
|  9.15883759E10|    2|
|  2.409759955E9|    2|
|1.2514478995E10|    2|
|  9.403318961E9|    2|
|5.2533867004E10|    2|
|  4.382618959E9|    2|
|  7.107035975E9|    2|
|  4.042480985E9|    2|
|7.8940524934E10|    2|
|2.4841323953E10|    2|
|  8.152253936E9|    2|
| 8.617561092E10|    2|
|4.2058686934E10|    2|
|  2.571880969E9|    2|
|  2.051876258E9|    2|
+---------------+-----+
only showing top 20 rows

6437
+-------------------+--------------+--------------------+-------------+-----------+----+-----+------------+--------------------+-----------------+--------------+------------+-------------+-------------+-----------------+-------------+-----------------+------+--------+-----------------------------+--------------------------+---------+-------------------+--------------+---------------------+-

In [33]:
duplos_cpf = df2.groupBy("CPF_CNPJ").count().filter("count > 1").select("CPF_CNPJ")
duplos_lista = [row["CPF_CNPJ"] for row in duplos_cpf.collect()]
df2_pivot=df2.filter(col("CPF_CNPJ").isin(duplos_lista)).select("CPF_CNPJ", "NIVEL_DE_RISCO")
df2_pivot.show()
## salvar arquivo
df2_pivot = df2_pivot.toPandas() # pandas
df2_pivot.to_excel("Duplicados.xlsx", index=False)


+-----------+--------------+
|   CPF_CNPJ|NIVEL_DE_RISCO|
+-----------+--------------+
|93962126953|            N6|
|93962126953|            N7|
|66171318949|            N6|
|66171318949|            N7|
|  776776096|            N6|
|  776776096|            N7|
|62761277015|            N6|
|62761277015|            N7|
|91588375900|            N6|
|91588375900|            N7|
|90514890053|            N6|
|90514890053|            N7|
| 2311342908|            N7|
| 2311342908|            N6|
|59129000297|            N6|
|59129000297|            N7|
|59561459949|            N6|
|59561459949|            N7|
|24870765004|            N6|
|24870765004|            N7|
+-----------+--------------+
only showing top 20 rows



In [ ]:

from pyspark.sql.window import Window
window_spec = Window.partitionBy("CPF_CNPJ").orderBy("NIVEL_DE_RISCO")
df2_pivot = df2_pivot.withColumn("row_num", row_number().over(window_spec))
df2_pivot = df2_pivot.groupBy("CPF_CNPJ").pivot("row_num").agg(first("NIVEL_DE_RISCO"))
df2_pivot = df2_pivot.withColumn("1", regexp_extract("1", r"(\d+)", 1)) \
             .withColumn("2", regexp_extract("2", r"(\d+)", 1))
df2_pivot=df2_pivot.withColumn("diff",col("1")-col("2"))
df2_pivot.groupBy(col("2")).count().show()

### Frequência

In [35]:
# números de linhas
total = df2.count()
print(total)

540814


In [36]:
## Variável string
df2.groupBy("ANO_MES_MOVIMENTO").count().show()

+-------------------+------+
|  ANO_MES_MOVIMENTO| count|
+-------------------+------+
|2025-04-01 00:00:00|540813|
|               NULL|     1|
+-------------------+------+



In [9]:
# tabela de frequencia
col_cat = [f.name for f in df2.schema.fields if f.dataType.simpleString() in ["string", "boolean"]]
col_cat=['DATA_MOVIMENTO', 'TIPO_PESSOA', 'SEXO', 'ESTADO_CIVIL', 'GRAU_ESCOLARIDADE', 'NATUREZA_OCUPACAO', 'NACIONALIDADE', 'UF_NASCIMENTO', 'CIDADE_NASCIMENTO', 'UF_RESIDENCIA', 'CIDADE_RESIDENCIA', 'FILIAL', 'BLACKLIST', 'DATA_ABERTURA_CONTA', 'SITUACAO_CONTA', 'MOTIVO_SITUACAO_CONTA', 'CANAL_ORIGEM', 'NIVEL_DE_RISCO', 'ORIGEM_SCORE', 'DATA_ULTIMA_ALTERACAO_LIMITE', 'ORIGEM_ALTERACAO_LIMITE', 'SITUACAO_FATURA', 'HISTORICO_COBRANCA']

# loop
for col_nome in col_cat:
    print(f"\n### Frequência para: {col_nome} ###")
    df2.groupBy(col_nome).count().orderBy("count", ascending=False).show(truncate=False)


### Frequência para: DATA_MOVIMENTO ###
+--------------+------+
|DATA_MOVIMENTO|count |
+--------------+------+
|23/04/2025    |540813|
|SC            |1     |
+--------------+------+


### Frequência para: TIPO_PESSOA ###
+-----------+------+
|TIPO_PESSOA|count |
+-----------+------+
|PF         |540811|
|NULL       |1     |
|9689020900 |1     |
|96356022949|1     |
+-----------+------+


### Frequência para: SEXO ###
+----+------+
|SEXO|count |
+----+------+
|F   |284935|
|M   |255413|
|    |463   |
|PF  |2     |
|2024|1     |
+----+------+


### Frequência para: ESTADO_CIVIL ###
+-------------+------+
|ESTADO_CIVIL |count |
+-------------+------+
|CASADO       |232295|
|SOLTEIRO     |145520|
|UNIAO ESTAVEL|53440 |
|DIVORCIADO   |38313 |
|VIUVO        |36386 |
|NULL         |34857 |
|NAO          |1     |
|24           |1     |
|75           |1     |
+-------------+------+


### Frequência para: GRAU_ESCOLARIDADE ###
+------------------------------+------+
|GRAU_ESCOLARIDADE        

In [10]:
## Variável string
df2.groupBy("NIVEL_DE_RISCO").count().orderBy("NIVEL_DE_RISCO").show(300,truncate=False)

+--------------+-----+
|NIVEL_DE_RISCO|count|
+--------------+-----+
|NULL          |718  |
|602,78        |1    |
|A1            |72360|
|A10           |6318 |
|A11           |5386 |
|A12           |4664 |
|A13           |4267 |
|A14           |3859 |
|A15           |3509 |
|A16           |3164 |
|A17           |2873 |
|A18           |2776 |
|A19           |2942 |
|A2            |47040|
|A20           |19163|
|A21           |77   |
|A3            |29976|
|A4            |21121|
|A5            |16355|
|A6            |13265|
|A7            |10190|
|A8            |8554 |
|A9            |7234 |
|Fisico        |2    |
|INTERNO       |1    |
|N1            |10505|
|N10           |17126|
|N11           |19163|
|N12           |16110|
|N13           |17127|
|N14           |7593 |
|N15           |3943 |
|N16           |1304 |
|N17           |215  |
|N18           |32   |
|N19           |16   |
|N2            |9228 |
|N20           |5    |
|N3            |20305|
|N4            |20133|
|N5        

In [11]:
## Categorias
df2.select("NIVEL_DE_RISCO").distinct().show(100,truncate=False)

+--------------+
|NIVEL_DE_RISCO|
+--------------+
|A9            |
|A6            |
|N18           |
|N10           |
|N19           |
|A19           |
|A16           |
|A21           |
|A17           |
|A2            |
|A8            |
|A18           |
|N3            |
|N15           |
|N12           |
|A11           |
|N16           |
|A10           |
|N13           |
|N11           |
|A20           |
|N17           |
|A4            |
|A15           |
|N7            |
|N8            |
|A14           |
|A7            |
|N2            |
|N5            |
|N4            |
|N14           |
|A5            |
|N6            |
|A12           |
|N1            |
|N9            |
|A13           |
|A3            |
|N20           |
|A1            |
|602,78        |
|INTERNO       |
|Fisico        |
|NULL          |
+--------------+



In [38]:
from functools import reduce

num_cols_df2=len(df2.columns)
cols_to_check = df2.columns
df2_com_nulos=df2.withColumn("%_nulos",
                             round(reduce(lambda a,b: a+b,
                                      [when(col(c).isNull(),1).otherwise(0) for c in cols_to_check])
                                      / lit(num_cols_df2)*100,4)
                             )

df2_com_nulos_order=df2_com_nulos.orderBy("%_nulos",ascending=False) # Ordem crescente
df2_com_nulos_order.show()
#df2_com_nulos_order.printSchema()

+-------------------+--------------+--------------------+---------------+-----------+----+-----+------------+--------------------+--------------------+--------------+------------+-------------+-------------+-----------------+-------------+--------------------+------+--------+-----------------------------+--------------------------+---------+-------------------+--------------+---------------------+------------+--------------+----------+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+---------------+------------------+-------+
|  ANO_MES_MOVIMENTO|DATA_MOVIMENTO|                NOME|       cpf_cnpj|TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|   GRAU_ESCOLARIDADE|   NATUREZA_OCUPACAO|RENDA_DIGITADA|RENDA_VALIDA|NACIONALIDADE|UF_NASCIM

In [39]:
#Dados errados
dados_error=df2.filter((col("SEXO") == "PF") | (col("DATA_MOVIMENTO") == "SC") | (col("BLACKLIST") == "Bloqueada"))
dados_error.show()

+-------------------+--------------+--------------------+-------------+-----------+----+-----+------------+-------------------+--------------------+-----------------+------------+-------------+-------------+-----------------+------------------+-----------------+----------+--------+-----------------------------+--------------------------+---------+-------------------+--------------+---------------------+------------+--------------+----------+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+---------------+------------------+
|  ANO_MES_MOVIMENTO|DATA_MOVIMENTO|                NOME|     cpf_cnpj|TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|  GRAU_ESCOLARIDADE|   NATUREZA_OCUPACAO|   RENDA_DIGITADA|RENDA_VALIDA|NACIONALIDADE|UF_NASCIMEN

In [40]:
# Excluindo linhas erradas
df2_limpo = df2_com_nulos_order.filter(
    ~(
        (col("SEXO") == "PF") |
        (col("DATA_MOVIMENTO") == "SC") |
        (col("BLACKLIST") == "Bloqueada") |
        (col("%_nulos")>50) 
    )
)
df2_limpo.count()

540809

In [16]:
# tabela de frequencia
col_cat = [f.name for f in df2_limpo.schema.fields if f.dataType.simpleString() in ["string", "boolean"]]
col_cat = ['DATA_MOVIMENTO', 'TIPO_PESSOA', 'SEXO', 'ESTADO_CIVIL', 'GRAU_ESCOLARIDADE', 'NATUREZA_OCUPACAO', 'NACIONALIDADE', 'UF_NASCIMENTO', 'CIDADE_NASCIMENTO', 'UF_RESIDENCIA', 'CIDADE_RESIDENCIA', 'FILIAL', 'BLACKLIST', 'DATA_ABERTURA_CONTA', 'SITUACAO_CONTA', 'MOTIVO_SITUACAO_CONTA', 'CANAL_ORIGEM', 'NIVEL_DE_RISCO', 'ORIGEM_SCORE', 'DATA_ULTIMA_ALTERACAO_LIMITE', 'ORIGEM_ALTERACAO_LIMITE', 'SITUACAO_FATURA', 'HISTORICO_COBRANCA']

# loop
for col_nome in col_cat:
    print(f"\n### Frequência para: {col_nome} ###")
    df2_limpo.groupBy(col_nome).count().orderBy("count", ascending=False).show(truncate=False)


### Frequência para: DATA_MOVIMENTO ###
+--------------+------+
|DATA_MOVIMENTO|count |
+--------------+------+
|23/04/2025    |540809|
+--------------+------+


### Frequência para: TIPO_PESSOA ###
+-----------+------+
|TIPO_PESSOA|count |
+-----------+------+
|PF         |540809|
+-----------+------+


### Frequência para: SEXO ###
+----+------+
|SEXO|count |
+----+------+
|F   |284934|
|M   |255412|
|    |463   |
+----+------+


### Frequência para: ESTADO_CIVIL ###
+-------------+------+
|ESTADO_CIVIL |count |
+-------------+------+
|CASADO       |232294|
|SOLTEIRO     |145519|
|UNIAO ESTAVEL|53440 |
|DIVORCIADO   |38313 |
|VIUVO        |36386 |
|NULL         |34857 |
+-------------+------+


### Frequência para: GRAU_ESCOLARIDADE ###
+------------------------------+------+
|GRAU_ESCOLARIDADE             |count |
+------------------------------+------+
|ENSINO MEDIO 2� GRAU COM      |339877|
|NULL                          |60434 |
|ENSINO FUNDAMENTA 1� GRAU COMP|42747 |
|ENSINO FU

In [17]:
# armazenar as tabelas
tabelas_frequencia = {}

for col_nome in col_cat:
    df_freq = df2_limpo.groupBy(col_nome).count().orderBy("count", ascending=False)
    tabelas_frequencia[col_nome] = df_freq

# Excel
with pd.ExcelWriter("tab_freq_base_cartoes.xlsx", engine="xlsxwriter") as writer:
    for nome, df_spark in tabelas_frequencia.items():
        df_pandas = df_spark.toPandas() # pandas
        aba = nome[:31] if len(nome) > 31 else nome # para variáveis com nomes grandes, mas não é o caso 
        df_pandas.to_excel(writer, sheet_name=aba, index=False)


In [ ]:
# Contar distintos
#df2.select(countDistinct("CPF_CNPJ")).show()
# loop ## não considera o valor nulo como distinto
df2_limpo.select([
    countDistinct(col(c)).alias(f"{c}_distinct") for c in df2_limpo.columns
]).show(truncate=False)

+--------------------------+-----------------------+-------------+-----------------+--------------------+-------------+--------------+---------------------+--------------------------+--------------------------+-----------------------+---------------------+----------------------+----------------------+--------------------------+----------------------+--------------------------+---------------+-----------------+--------------------------------------+-----------------------------------+------------------+----------------------------+-----------------------+------------------------------+---------------------+-----------------------+--------------+---------------------+---------------------+-----------------------+---------------------+----------------------+-------------------------------+---------------------------------+-------------------------------+--------------------------------+--------------------------------+----------------------------------+--------------------------------+---

In [20]:
### Contar nulos
#df2.select(count(when(col("ANO_MES_MOVIMENTO").isNull(), 1)).alias("nulos")).show()

#loop
df2_limpo.select([
    count(when(col(c).isNull(), 1)).alias(f"{c}_nulos") for c in df2_limpo.columns
]).show(truncate=False)

+-----------------------+--------------------+----------+--------------+-----------------+----------+-----------+------------------+-----------------------+-----------------------+--------------------+------------------+-------------------+-------------------+-----------------------+-------------------+-----------------------+------------+--------------+-----------------------------------+--------------------------------+---------------+-------------------------+--------------------+---------------------------+------------------+--------------------+-----------+------------------+------------------+--------------------+------------------+-------------------+----------------------------+------------------------------+----------------------------+-----------------------------+-----------------------------+-------------------------------+-----------------------------+------------------------------+----------------------------------+-----------------------------+---------------------+-----

In [26]:
#Mostrar os nulos
nulo_res=df2_limpo.filter(col("ESTADO_CIVIL").isNull())
nulo_res.show(truncate=False)
## salvar arquivo
nulo_res_p = nulo_res.toPandas() # pandas
nulo_res_p.to_excel("nulo_res_p.xlsx", index=False)

+-------------------+--------------+--------------------------------+-----------+-----------+----+-----+------------+------------------------------+------------------------------+--------------+------------+-------------+-------------+-----------------+-------------+--------------------+------+--------+-----------------------------+--------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+---------------+------------------+
|ANO_MES_MOVIMENTO  |DATA_MOVIMENTO|NOME                            |CPF_CNPJ   |TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|GRAU_ESCOLARIDADE             |NATUREZA_OCUPACAO             |RENDA_DI

In [27]:
#Filtro
df2_limpo.filter((col("ORIGEM_SCORE") == "0")).show(1000)

+-------------------+--------------+--------------------+-----------+-----------+----+-----+------------+-----------------+--------------------+--------------+------------+-------------+-------------+-----------------+-------------+--------------------+------+--------+-----------------------------+--------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+---------------+------------------+
|  ANO_MES_MOVIMENTO|DATA_MOVIMENTO|                NOME|   CPF_CNPJ|TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|GRAU_ESCOLARIDADE|   NATUREZA_OCUPACAO|RENDA_DIGITADA|RENDA_VALIDA|NACIONALIDADE|UF_NASCIMENTO|CIDADE_NASCIMENTO|UF_R

## Base Contratos Cartão

In [40]:
## Base cartões 

arquivos_cartao = [
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2015.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2016.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2017.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2018.csv",    
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2019.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2020.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2021.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2022.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2023.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2024.csv",
    "C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base Contratos cartão/contratos_2025.csv"]

df_cartao = spark.read.csv(arquivos_cartao,sep=";", header=True, inferSchema=True)

In [41]:
df_cartao.count()

2292529

In [43]:
# info
df_cartao.printSchema()

root
 |-- ANO_MES_MOVIMENTO: timestamp (nullable = true)
 |-- CPF_CNPJ: double (nullable = true)
 |-- NUMERO_CONTRATO: integer (nullable = true)
 |-- DATA_CONTRATO: date (nullable = true)
 |-- MODALIDADE: integer (nullable = true)
 |-- DIAS_CARENCIA: integer (nullable = true)
 |-- TAXA_JUROS: string (nullable = true)
 |-- CET: string (nullable = true)
 |-- SALDO_CONTABIL: string (nullable = true)
 |-- DIAS_MAIOR_ATRASO_ABERTO: integer (nullable = true)
 |-- DIAS_MAIOR_ATRASO: integer (nullable = true)
 |-- MEDIA_ATRASO: integer (nullable = true)
 |-- QTD_PARCELAS: integer (nullable = true)
 |-- QTD_PARCELAS_ABERTAS: integer (nullable = true)
 |-- QTD_PARCELAS_PAGAS: integer (nullable = true)
 |-- VALOR_PAGO: string (nullable = true)
 |-- JUROS_PAGOS_ATRASO: integer (nullable = true)
 |-- VALOR_PARCELA: string (nullable = true)
 |-- DATA_VENCIMENTO_PROX_PARCELA: string (nullable = true)
 |-- DATA_VENCIMENTO_ULTIMA_PARCELA: string (nullable = true)
 |-- VALOR_CALCULADO: string (nullable 

In [44]:
# números de linhas
df_cartao.show()

+-------------------+---------------+---------------+-------------+----------+-------------+----------+------+--------------+------------------------+-----------------+------------+------------+--------------------+------------------+----------+------------------+-------------+----------------------------+------------------------------+---------------+--------------+---------------+----------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------

In [45]:
df_cartao.filter(col("cpf_cnpj") == "56595166968").show() # filtro por Cpf

+-------------------+---------------+---------------+-------------+----------+-------------+----------+----+--------------+------------------------+-----------------+------------+------------+--------------------+------------------+----------+------------------+-------------+----------------------------+------------------------------+---------------+--------------+---------------+----------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+-------------------

In [46]:
df_cartao.filter(col("JUROS_PAGOS_ATRASO") > 0).show() # filtro

+-----------------+--------+---------------+-------------+----------+-------------+----------+---+--------------+------------------------+-----------------+------------+------------+--------------------+------------------+----------+------------------+-------------+----------------------------+------------------------------+---------------+--------------+---------------+----------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------

In [47]:
## Variável string
df_cartao.groupBy("ANO_MES_MOVIMENTO").count().show() # agrupar por categoria

+-------------------+-------+
|  ANO_MES_MOVIMENTO|  count|
+-------------------+-------+
|2025-06-01 00:00:00|2292529|
+-------------------+-------+



In [49]:
# tabela de frequencia
col_cat_df3 = [f.name for f in df_cartao.schema.fields if f.dataType.simpleString() in ["string", "boolean"]]
col_cat_df3=["ANO_MES_MOVIMENTO",
"DATA_CONTRATO",
"MODALIDADE",
"DATA_VENCIMENTO_PROX_PARCELA",
"DATA_VENCIMENTO_ULTIMA_PARCELA",
"FLAG_RENEGOCIACAO",
"_c48"
]

# loop
for col_nome in col_cat_df3:
    print(f"\n### Frequência para: {col_nome} ###")
    df_cartao.groupBy(col_nome).count().orderBy("count", ascending=False).show(truncate=False)


### Frequência para: ANO_MES_MOVIMENTO ###
+-------------------+-------+
|ANO_MES_MOVIMENTO  |count  |
+-------------------+-------+
|2025-06-01 00:00:00|2292529|
+-------------------+-------+


### Frequência para: DATA_CONTRATO ###
+-------------+-----+
|DATA_CONTRATO|count|
+-------------+-----+
|2022-12-12   |4582 |
|2022-09-12   |3912 |
|2022-07-11   |3859 |
|2022-04-11   |3799 |
|2019-11-11   |3687 |
|2021-10-11   |3682 |
|2022-10-10   |3529 |
|2019-08-12   |3411 |
|2024-02-14   |3397 |
|2021-07-12   |3361 |
|2023-04-10   |3306 |
|2022-01-10   |3291 |
|2021-05-10   |3282 |
|2020-02-10   |3252 |
|2022-03-10   |3237 |
|2022-11-10   |3223 |
|2021-04-12   |3213 |
|2019-06-10   |3210 |
|2023-09-11   |3195 |
|2019-03-11   |3170 |
+-------------+-----+
only showing top 20 rows


### Frequência para: MODALIDADE ###
+----------+------+
|MODALIDADE|count |
+----------+------+
|2         |797496|
|3         |769868|
|4         |353688|
|5         |259105|
|1         |112372|
+----------+--

In [52]:
# Contar distintos
#df3.select(countDistinct("CPF_CNPJ")).show()

# loop ## não considera o valor nulo como distinto
df_cartao.select([
    countDistinct(col(c)).alias(f"{c}_distinct") for c in df_cartao.columns
]).show(truncate=False)

+--------------------------+-----------------+------------------------+----------------------+-------------------+----------------------+-------------------+------------+-----------------------+---------------------------------+--------------------------+---------------------+---------------------+-----------------------------+---------------------------+-------------------+---------------------------+----------------------+-------------------------------------+---------------------------------------+------------------------+-----------------------+------------------------+-------------------------+--------------------------+--------------------------------------+------------------------------------------+------------------------------------------+-------------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------------+----------------------------------------------+---------------

In [53]:
### Contar nulos
#df3.select(count(when(col("ANO_MES_MOVIMENTO").isNull(), 1)).alias("nulos")).show()

#loop
df_cartao.select([
    count(when(col(c).isNull(), 1)).alias(f"{c}_nulos") for c in df_cartao.columns
]).show(truncate=False)

+-----------------------+--------------+---------------------+-------------------+----------------+-------------------+----------------+---------+--------------------+------------------------------+-----------------------+------------------+------------------+--------------------------+------------------------+----------------+------------------------+-------------------+----------------------------------+------------------------------------+---------------------+--------------------+---------------------+----------------------+-----------------------+-----------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+--------------------------

## Join Base contratos Cartão e Base Cadastral

In [ ]:
#df2_limpo & df3

In [170]:
df2_limpo_ren = df2_limpo.withColumnRenamed("CPF_CNPJ","CPF_CNPJ_df2") 
df_cartao_join=df3.join(df2_limpo_ren,df3['CPF_CNPJ'] == df2_limpo_ren["CPF_CNPJ_df2"],"inner") 
df_cartao_join.show()

+-----------------+--------+---------------+-------------+----------+-------------+----------+-------+--------------+-----------------+------------+------------+--------------------+------------------+----------+------------------+-------------+----------------------------+------------------------------+---------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+--------------------------

In [171]:
print(df_cartao_join.count())
print(df3.count())
print(df2_limpo_ren.count())

1049037
1048578
540809


In [172]:
df3.agg(countDistinct("cpf_cnpj").alias("Valores_distintos_df3")).show()
df2_limpo_ren.agg(countDistinct("CPF_CNPJ_df2").alias("Valores_distintos_df2_limpo_ren")).show()
df_cartao_join.agg(countDistinct("cpf_cnpj").alias("Valores_distintos_df_cartao_join")).show()

+---------------------+
|Valores_distintos_df3|
+---------------------+
|               247062|
+---------------------+

+-------------------------------+
|Valores_distintos_df2_limpo_ren|
+-------------------------------+
|                         534373|
+-------------------------------+

+--------------------------------+
|Valores_distintos_df_cartao_join|
+--------------------------------+
|                          247059|
+--------------------------------+



In [173]:
df3_nao_enc = df3.join(df2_limpo_ren, df3["CPF_CNPJ"] == df2_limpo_ren["CPF_CNPJ_df2"], "left_anti")
df3_nao_enc.show()
print(df3_nao_enc.count())

+--------------------+-----------------+---------------+-------------+----------+-------------+----------+----+--------------+-----------------+------------+------------+--------------------+------------------+----------+------------------+-------------+----------------------------+------------------------------+---------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+-----------------

In [174]:
df3_nao_enc.select(countDistinct(col("CPF_CNPJ"))).show()
df3_nao_enc.select("CPF_CNPJ").distinct().orderBy("CPF_CNPJ",ascending=True).show()


+------------------------+
|count(DISTINCT CPF_CNPJ)|
+------------------------+
|                       3|
+------------------------+

+-----------------+
|         CPF_CNPJ|
+-----------------+
|             NULL|
|      29222621972|
|      96356022949|
|L.0141 KAD (0141)|
+-----------------+



In [ ]:
## salvar arquivo
#df_nao_enc_p = df_nao_enc.toPandas() # pandas
#df_nao_enc_p.to_excel("Cpf_Carne_n_enc.xlsx", index=False)


## Join Base contratos Carnê e Base Cadastral

In [ ]:
#df2_limpo & df

In [165]:
df2_limpo_ren = df2_limpo.withColumnRenamed("CPF_CNPJ","CPF_CNPJ_df2") 
df_carne_join=df.join(df2_limpo_ren,df['CPF_CNPJ'] == df2_limpo_ren["CPF_CNPJ_df2"],"inner") 
df_carne_join.show()

+-------+---------+--------+-----------+-------------+----------+--------------------+-------------+---------+----+--------------+-----------------+------------------------+------------+------------+--------------------+------------------+-------------+----------+-----------------+-------------+----------------------------+------------------------------+---------------+--------------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+--------------------------

In [152]:
print(df_carne_join.count())
print(df.count())
print(df2_limpo_ren.count())

817778
837801
540809


In [153]:
df.agg(countDistinct("cpf_cnpj").alias("Valores_distintos_df")).show()
df2_limpo_ren.agg(countDistinct("CPF_CNPJ_df2").alias("Valores_distintos_df2_limpo_ren")).show()
df_carne_join.agg(countDistinct("cpf_cnpj").alias("Valores_distintos_df_carne_join")).show()

+--------------------+
|Valores_distintos_df|
+--------------------+
|              300679|
+--------------------+

+-------------------------------+
|Valores_distintos_df2_limpo_ren|
+-------------------------------+
|                         534373|
+-------------------------------+

+-------------------------------+
|Valores_distintos_df_carne_join|
+-------------------------------+
|                         289071|
+-------------------------------+



In [162]:
df_nao_enc = df.join(df2_limpo_ren, df["CPF_CNPJ"] == df2_limpo_ren["CPF_CNPJ_df2"], "left_anti")
df_nao_enc.show()
print(df_nao_enc.count())

+-------+---------+--------+-----------+-------------+----------+--------------------+-------------+---------+----+--------------+-----------------+------------------------+------------+------------+--------------------+------------------+-------------+----------+-----------------+-------------+----------------------------+------------------------------+---------------+--------------------+-----------------+-----------------------------+---------------------------------+---------------------------------+----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+------------------------------------+--------------------------------+---------------------------------+---------------------------------+---------------------------------+----------------------------------+--------------------------

In [163]:
df_nao_enc.select(countDistinct(col("CPF_CNPJ"))).show()
df_nao_enc.select("CPF_CNPJ").distinct().orderBy("CPF_CNPJ",ascending=True).show()


+------------------------+
|count(DISTINCT CPF_CNPJ)|
+------------------------+
|                   11608|
+------------------------+

+-----------+
|   CPF_CNPJ|
+-----------+
|   929085.0|
|  2565013.0|
|  3641937.0|
|  3957900.0|
|  4199901.0|
|  4270983.0|
|  4438914.0|
|  4530993.0|
|  4579909.0|
|  4640900.0|
|  5621909.0|
|  6192998.0|
|  6631908.0|
|  7551924.0|
|  7573901.0|
|  7601964.0|
|  7607903.0|
|  8086010.0|
|1.0202935E7|
|1.1101954E7|
+-----------+
only showing top 20 rows



In [ ]:
## salvar arquivo
#df_nao_enc_p = df_nao_enc.toPandas() # pandas
#df_nao_enc_p.to_excel("Cpf_Carne_n_enc.xlsx", index=False)


# Base Cadastro NOVA

In [13]:
## Base cartões

df2=spark.read.csv("C:/Users/fernando.monteiro/OneDrive - BHS/KAB/Base Behavior/Base cartões/Base_clientes_09.06.25.csv",header=True,inferSchema=True, sep=";") 

'''#Tipos
tiposdf2 = {
    "cpf_cnpj": "double"#,
}

# conversão
for col_nome, tipo in tiposdf2.items():
    df2 = df2.withColumn(col_nome, col(col_nome).cast(tipo))'''

df2.show()

+-------------------+--------------+--------------------+-----------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------+------------+-------------+-------------+------------------+-------------+--------------------+------+--------+-----------------------------+------------------------------+-----------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+--------------------+---------------+------------------+
|  ANO_MES_MOVIMENTO|DATA_MOVIMENTO|                NOME|   CPF_CNPJ|TIPO_PESSOA|SEXO|IDADE| ESTADO_CIVIL|   GRAU_ESCOLA

In [14]:
# info
df2.printSchema()

root
 |-- ANO_MES_MOVIMENTO: timestamp (nullable = true)
 |-- DATA_MOVIMENTO: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- CPF_CNPJ: long (nullable = true)
 |-- TIPO_PESSOA: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: integer (nullable = true)
 |-- ESTADO_CIVIL: string (nullable = true)
 |-- GRAU_ESCOLARIDADE: string (nullable = true)
 |-- PROFISSAO: string (nullable = true)
 |-- CARGO: string (nullable = true)
 |-- NATUREZA_OCUPACAO: string (nullable = true)
 |-- RENDA_DIGITADA: string (nullable = true)
 |-- RENDA_VALIDA: string (nullable = true)
 |-- NACIONALIDADE: string (nullable = true)
 |-- UF_NASCIMENTO: string (nullable = true)
 |-- CIDADE_NASCIMENTO: string (nullable = true)
 |-- UF_RESIDENCIA: string (nullable = true)
 |-- CIDADE_RESIDENCIA: string (nullable = true)
 |-- FILIAL: integer (nullable = true)
 |-- REGIONAL: string (nullable = true)
 |-- TEMPO_RELACIONAMENTO_KREDILIG: integer (nullable = true)
 |-- TEMPO_INATIVIDADE

### Duplos

In [ ]:
## Removendo dados duplicados -- tudo certo!!
duplos=df2.groupBy(col("CPF_CNPJ")).count().filter(col("count")>1)
duplos.show()
print(duplos.count())
df2.filter(col("CPF_CNPJ")=="1893026078").show()


+--------+-----+
|CPF_CNPJ|count|
+--------+-----+
+--------+-----+

0
+-------------------+--------------+--------------------+----------+-----------+----+-----+------------+--------------------+---------+-------+--------------------+--------------+------------+-------------+-------------+-----------------+-------------+-----------------+------+--------+-----------------------------+------------------------------+-----------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+-------------------+---------------+------------------+
|  ANO_MES_MOVIMENTO|DATA_MOVIMENTO|                NOME|  CPF_CNPJ|TIPO_PESSOA|

In [ ]:
duplos_cpf = df2.groupBy("CPF_CNPJ").count().filter("count > 1").select("CPF_CNPJ")
duplos_lista = [row["CPF_CNPJ"] for row in duplos_cpf.collect()]
df2_pivot=df2.filter(col("CPF_CNPJ").isin(duplos_lista)).select("CPF_CNPJ", "NIVEL_DE_RISCO")
df2_pivot.show()
## salvar arquivo
#df2_pivot = df2_pivot.toPandas() # pandas
#df2_pivot.to_excel("Duplicados.xlsx", index=False)


+--------+--------------+
|CPF_CNPJ|NIVEL_DE_RISCO|
+--------+--------------+
+--------+--------------+



In [ ]:

from pyspark.sql.window import Window
window_spec = Window.partitionBy("CPF_CNPJ").orderBy("NIVEL_DE_RISCO")
df2_pivot = df2_pivot.withColumn("row_num", row_number().over(window_spec))
df2_pivot = df2_pivot.groupBy("CPF_CNPJ").pivot("row_num").agg(first("NIVEL_DE_RISCO"))
df2_pivot = df2_pivot.withColumn("1", regexp_extract("1", r"(\d+)", 1)) \
             .withColumn("2", regexp_extract("2", r"(\d+)", 1))
df2_pivot=df2_pivot.withColumn("diff",col("1")-col("2"))
df2_pivot.groupBy(col("2")).count().show()

### Frequência

In [11]:
# números de linhas
total = df2.count()
print(total)

541772


In [12]:
## Variável string
df2.groupBy("ANO_MES_MOVIMENTO").count().show()

+-------------------+------+
|  ANO_MES_MOVIMENTO| count|
+-------------------+------+
|2025-06-01 00:00:00|541772|
+-------------------+------+



In [15]:
# tabela de frequencia
col_cat = [f.name for f in df2.schema.fields if f.dataType.simpleString() in ["string", "boolean"]]
col_cat=[
        "ANO_MES_MOVIMENTO",
        "DATA_MOVIMENTO",
        "TIPO_PESSOA",
        "SEXO",
        "ESTADO_CIVIL",
        "GRAU_ESCOLARIDADE",
        "PROFISSAO",
        "CARGO",
        "NATUREZA_OCUPACAO",
        "NACIONALIDADE",
        "UF_NASCIMENTO",
        "CIDADE_NASCIMENTO",
        "UF_RESIDENCIA",
        "CIDADE_RESIDENCIA",
        "FILIAL",
        "REGIONAL",
        "BLACKLIST",
        "DATA_ABERTURA_CONTA",
        "SITUACAO_CONTA",
        "MOTIVO_SITUACAO_CONTA",
        "CANAL_ORIGEM",
        "NIVEL_DE_RISCO",
        "ORIGEM_SCORE",
        "DATA_ULTIMA_ALTERACAO_LIMITE",
        "ORIGEM_ALTERACAO_LIMITE",
        "HISTORICO_ALTERACAO",
        "SITUACAO_FATURA",
        "HISTORICO_COBRANCA"
        ]

# loop
for col_nome in col_cat:
    print(f"\n### Frequência para: {col_nome} ###")
    df2.groupBy(col_nome).count().orderBy("count", ascending=False).show(truncate=False)


### Frequência para: ANO_MES_MOVIMENTO ###
+-------------------+------+
|ANO_MES_MOVIMENTO  |count |
+-------------------+------+
|2025-06-01 00:00:00|541772|
+-------------------+------+


### Frequência para: DATA_MOVIMENTO ###
+--------------+------+
|DATA_MOVIMENTO|count |
+--------------+------+
|09/06/2025    |541772|
+--------------+------+


### Frequência para: TIPO_PESSOA ###
+-----------+------+
|TIPO_PESSOA|count |
+-----------+------+
|PF         |541772|
+-----------+------+


### Frequência para: SEXO ###
+----+------+
|SEXO|count |
+----+------+
|F   |285517|
|M   |255813|
|    |442   |
+----+------+


### Frequência para: ESTADO_CIVIL ###
+-------------+------+
|ESTADO_CIVIL |count |
+-------------+------+
|CASADO       |232911|
|SOLTEIRO     |146254|
|UNIAO ESTAVEL|53485 |
|DIVORCIADO   |38350 |
|VIUVO        |36495 |
|NULL         |34277 |
+-------------+------+


### Frequência para: GRAU_ESCOLARIDADE ###
+------------------------------+------+
|GRAU_ESCOLARIDADE  

In [16]:
## Variável string
df2.groupBy("NIVEL_DE_RISCO").count().orderBy("NIVEL_DE_RISCO").show(300,truncate=False)

+--------------+-----+
|NIVEL_DE_RISCO|count|
+--------------+-----+
|NULL          |2    |
|A1            |72166|
|A10           |6333 |
|A11           |5394 |
|A12           |4666 |
|A13           |4273 |
|A14           |3855 |
|A15           |3529 |
|A16           |3179 |
|A17           |2885 |
|A18           |2778 |
|A19           |2944 |
|A2            |46867|
|A20           |19295|
|A21           |83   |
|A3            |29917|
|A4            |21176|
|A5            |16441|
|A6            |13331|
|A7            |10236|
|A8            |8566 |
|A9            |7254 |
|N1            |11949|
|N10           |17609|
|N11           |19601|
|N12           |16285|
|N13           |17237|
|N14           |7680 |
|N15           |3984 |
|N16           |1334 |
|N17           |234  |
|N18           |34   |
|N19           |18   |
|N2            |9695 |
|N20           |4    |
|N3            |20862|
|N4            |20860|
|N5            |20663|
|N6            |29380|
|N7            |15935|
|N8        

In [17]:
## Categorias
df2.select("NIVEL_DE_RISCO").distinct().show(100,truncate=False)

+--------------+
|NIVEL_DE_RISCO|
+--------------+
|A9            |
|A6            |
|N18           |
|N10           |
|A19           |
|A16           |
|A21           |
|A17           |
|A2            |
|A8            |
|A18           |
|N3            |
|N15           |
|N12           |
|A11           |
|N16           |
|A10           |
|N13           |
|N11           |
|A20           |
|N17           |
|A4            |
|A15           |
|N7            |
|N8            |
|A14           |
|A7            |
|N2            |
|N5            |
|N4            |
|N14           |
|A5            |
|N6            |
|A12           |
|N1            |
|N9            |
|A13           |
|A3            |
|N20           |
|A1            |
|N19           |
|NULL          |
+--------------+



In [18]:
from functools import reduce

num_cols_df2=len(df2.columns)
cols_to_check = df2.columns
df2_com_nulos=df2.withColumn("%_nulos",
                             round(reduce(lambda a,b: a+b,
                                      [when(col(c).isNull(),1).otherwise(0) for c in cols_to_check])
                                      / lit(num_cols_df2)*100,4)
                             )

df2_com_nulos_order=df2_com_nulos.orderBy("%_nulos",ascending=False) # Ordem crescente
df2_com_nulos_order.show()
#df2_com_nulos_order.printSchema()

+-------------------+--------------+--------------------+-----------+-----------+----+-----+------------+-----------------+--------------------+-----+--------------------+--------------+------------+-------------+-------------+-----------------+-------------+------------------+------+--------+-----------------------------+------------------------------+-----------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+-------------------+---------------+------------------+-------+
|  ANO_MES_MOVIMENTO|DATA_MOVIMENTO|                NOME|   CPF_CNPJ|TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|GRAU_ESCOLARIDADE|           P

In [19]:
#Dados errados
dados_error=df2.filter((col("SEXO") == "PF") | (col("DATA_MOVIMENTO") == "SC") | (col("BLACKLIST") == "Bloqueada"))
dados_error.show()

+-----------------+--------------+----+--------+-----------+----+-----+------------+-----------------+---------+-----+-----------------+--------------+------------+-------------+-------------+-----------------+-------------+-----------------+------+--------+-----------------------------+------------------------------+-----------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+-------------------+---------------+------------------+
|ANO_MES_MOVIMENTO|DATA_MOVIMENTO|NOME|CPF_CNPJ|TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|GRAU_ESCOLARIDADE|PROFISSAO|CARGO|NATUREZA_OCUPACAO|RENDA_DIGITADA|RENDA_VALIDA|NACIONALIDADE|U

In [20]:
# Excluindo linhas erradas
df2_limpo = df2_com_nulos_order.filter(
    ~(
        (col("SEXO") == "PF") |
        (col("DATA_MOVIMENTO") == "SC") |
        (col("BLACKLIST") == "Bloqueada") |
        (col("%_nulos")>50) 
    )
)
df2_limpo.count()

541772

In [21]:
# tabela de frequencia
col_cat = [f.name for f in df2_limpo.schema.fields if f.dataType.simpleString() in ["string", "boolean"]]
col_cat = ["ANO_MES_MOVIMENTO",
        "DATA_MOVIMENTO",
        "TIPO_PESSOA",
        "SEXO",
        "ESTADO_CIVIL",
        "GRAU_ESCOLARIDADE",
        "PROFISSAO",
        "CARGO",
        "NATUREZA_OCUPACAO",
        "NACIONALIDADE",
        "UF_NASCIMENTO",
        "CIDADE_NASCIMENTO",
        "UF_RESIDENCIA",
        "CIDADE_RESIDENCIA",
        "FILIAL",
        "REGIONAL",
        "BLACKLIST",
        "DATA_ABERTURA_CONTA",
        "SITUACAO_CONTA",
        "MOTIVO_SITUACAO_CONTA",
        "CANAL_ORIGEM",
        "NIVEL_DE_RISCO",
        "ORIGEM_SCORE",
        "DATA_ULTIMA_ALTERACAO_LIMITE",
        "ORIGEM_ALTERACAO_LIMITE",
        "HISTORICO_ALTERACAO",
        "SITUACAO_FATURA",
        "HISTORICO_COBRANCA"]

# loop
for col_nome in col_cat:
    print(f"\n### Frequência para: {col_nome} ###")
    df2_limpo.groupBy(col_nome).count().orderBy("count", ascending=False).show(truncate=False)


### Frequência para: ANO_MES_MOVIMENTO ###
+-------------------+------+
|ANO_MES_MOVIMENTO  |count |
+-------------------+------+
|2025-06-01 00:00:00|541772|
+-------------------+------+


### Frequência para: DATA_MOVIMENTO ###
+--------------+------+
|DATA_MOVIMENTO|count |
+--------------+------+
|09/06/2025    |541772|
+--------------+------+


### Frequência para: TIPO_PESSOA ###
+-----------+------+
|TIPO_PESSOA|count |
+-----------+------+
|PF         |541772|
+-----------+------+


### Frequência para: SEXO ###
+----+------+
|SEXO|count |
+----+------+
|F   |285517|
|M   |255813|
|    |442   |
+----+------+


### Frequência para: ESTADO_CIVIL ###
+-------------+------+
|ESTADO_CIVIL |count |
+-------------+------+
|CASADO       |232911|
|SOLTEIRO     |146254|
|UNIAO ESTAVEL|53485 |
|DIVORCIADO   |38350 |
|VIUVO        |36495 |
|NULL         |34277 |
+-------------+------+


### Frequência para: GRAU_ESCOLARIDADE ###
+------------------------------+------+
|GRAU_ESCOLARIDADE  

In [22]:
# armazenar as tabelas
tabelas_frequencia = {}

for col_nome in col_cat:
    df_freq = df2_limpo.groupBy(col_nome).count().orderBy("count", ascending=False)
    tabelas_frequencia[col_nome] = df_freq

# Excel
with pd.ExcelWriter("validacao_Cadastro.xlsx", engine="xlsxwriter") as writer:
    for nome, df_spark in tabelas_frequencia.items():
        df_pandas = df_spark.toPandas() # pandas
        aba = nome[:31] if len(nome) > 31 else nome # para variáveis com nomes grandes, mas não é o caso 
        df_pandas.to_excel(writer, sheet_name=aba, index=False)


In [23]:
# Contar distintos
#df2.select(countDistinct("CPF_CNPJ")).show()
# loop ## não considera o valor nulo como distinto
df2_limpo.select([
    countDistinct(col(c)).alias(f"{c}_distinct") for c in df2_limpo.columns
]).show(truncate=False)

+--------------------------+-----------------------+-------------+-----------------+--------------------+-------------+--------------+---------------------+--------------------------+------------------+--------------+--------------------------+-----------------------+---------------------+----------------------+----------------------+--------------------------+----------------------+--------------------------+---------------+-----------------+--------------------------------------+---------------------------------------+--------------------------------------+------------------+----------------------------+-----------------------+------------------------------+---------------------+-----------------------+--------------+---------------------+---------------------+-----------------------+---------------------+----------------------+-------------------------------+---------------------------------+-------------------------------+--------------------------------+---------------------------

In [24]:
### Contar nulos
#df2.select(count(when(col("ANO_MES_MOVIMENTO").isNull(), 1)).alias("nulos")).show()

#loop
df2_limpo.select([
    count(when(col(c).isNull(), 1)).alias(f"{c}_nulos") for c in df2_limpo.columns
]).show(truncate=False)

+-----------------------+--------------------+----------+--------------+-----------------+----------+-----------+------------------+-----------------------+---------------+-----------+-----------------------+--------------------+------------------+-------------------+-------------------+-----------------------+-------------------+-----------------------+------------+--------------+-----------------------------------+------------------------------------+-----------------------------------+---------------+-------------------------+--------------------+---------------------------+------------------+--------------------+-----------+------------------+------------------+--------------------+------------------+-------------------+----------------------------+------------------------------+----------------------------+-----------------------------+-----------------------------+-------------------------------+-----------------------------+------------------------------+------------------------

In [25]:
#Mostrar os nulos
nulo_res=df2_limpo.filter(col("ESTADO_CIVIL").isNull())
nulo_res.show(truncate=False)
## salvar arquivo
nulo_res_p = nulo_res.toPandas() # pandas
nulo_res_p.to_excel("nulo_res_p.xlsx", index=False)

+-------------------+--------------+------------------------------+-----------+-----------+----+-----+------------+-----------------+---------+----------+--------------------------+--------------+------------+-------------+-------------+-----------------+-------------+------------------+------+--------+-----------------------------+------------------------------+-----------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+-------------------+---------------+------------------+-------+
|ANO_MES_MOVIMENTO  |DATA_MOVIMENTO|NOME                          |CPF_CNPJ   |TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|GRAU_ESCOL

In [26]:
#Filtro
df2_limpo.filter((col("ORIGEM_SCORE") == "0")).show(1000)

+-----------------+--------------+----+--------+-----------+----+-----+------------+-----------------+---------+-----+-----------------+--------------+------------+-------------+-------------+-----------------+-------------+-----------------+------+--------+-----------------------------+------------------------------+-----------------------------+---------+-------------------+--------------+---------------------+------------+--------------+-----+------------+------------+--------------+------------+-------------+----------------------+------------------------+----------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------------+-----------------------+-------------------+---------------+------------------+-------+
|ANO_MES_MOVIMENTO|DATA_MOVIMENTO|NOME|CPF_CNPJ|TIPO_PESSOA|SEXO|IDADE|ESTADO_CIVIL|GRAU_ESCOLARIDADE|PROFISSAO|CARGO|NATUREZA_OCUPACAO|RENDA_DIGITADA|RENDA_VALIDA|NACIONA